# Diagnosis Prediction Tool: ETL Notebook

## Overview

#### This notebook contains the python code to populate a database that contains tables: 1. diseases; 2. diagnosis_samples.  The data for these tables comes from a kaggle dataset containing close to 5,000 samples of symptoms and diagnosis.

* The diseases data comes from two csv files (symptom_Description.csv and symptom_precaution.csv) found at:  https://www.kaggle.com/datasets/itachi9604/disease-symptom-description-dataset

* The diagnosis_sample data comes from two csv files (dataset.csv and Symptom-severity.csv) found at:  https://www.kaggle.com/datasets/itachi9604/disease-symptom-description-dataset

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy import Column, Integer, String, Float, Boolean, ForeignKey, UniqueConstraint, ForeignKeyConstraint
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy.ext.declarative import declarative_base
import config
import numpy as np

# locate config.py
# import sys
# sys.path.insert(0, "..")

# import the confidential information
# from config import pg_key, pg_db, pg_host, pg_port, pg_user

## Extract

* Each of the 4 csv files are loaded into dataframes
    * symptom_Description.csv to disease_desc_df
    * symptom_precaution.csv to disease_treat_df
    * dataset.csv to samples_df
    * Symptom-severity.csv to symp_sever_df

In [ ]:
# Extract symptom_Description.csv to disease_desc_df
filename = '../Resources/symptom_Description.csv'
disease_desc_df = pd.read_csv(filename)
disease_desc_df.head()

In [ ]:
# Extract symptom_precaution.csv to disease_treat_df
filename = '../Resources/symptom_precaution.csv'
disease_treat_df = pd.read_csv(filename)
disease_treat_df.head()

In [ ]:
# Extract dataset.csv to samples_df
filename = '../Resources/dataset.csv'
samples_df = pd.read_csv(filename)
samples_df.head()

In [ ]:
# Extract Symptom-severity.csv to symp_sever_df
filename = '../Resources/Symptom-severity.csv'
symp_sever_df = pd.read_csv(filename)
symp_sever_df.head()

## Transform

### 1. Transform Disease Data

In [ ]:
# Check that Disease column values are the same from treatment and description dataframes
disease_compare_df = pd.DataFrame()
disease_compare_df['desc_disease'] = disease_desc_df['Disease']
disease_compare_df['treat_disease'] = disease_treat_df['Disease']
disease_compare_df['Check'] =  disease_compare_df['treat_disease']==disease_compare_df['desc_disease']
disease_compare_df

In [ ]:
# Remove leading and trailing spaces from Disease columns in both dataframes
disease_desc_df['Disease'] = disease_desc_df['Disease'].str.strip()
disease_treat_df['Disease'] = disease_treat_df['Disease'].str.strip()

# fix misspelling in treatment file
disease_treat_df.loc[ disease_treat_df['Disease'] == 'Dimorphic hemmorhoids(piles)', 'Disease'] = 'Dimorphic hemorrhoids(piles)'

# Merge the disease_treat_df and disease_desc_df on the Disease field
disease_df = disease_desc_df.merge(disease_treat_df[['Disease', 'Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']], on=['Disease'])

# Rename columns to read treatments vs precautions
disease_df = disease_df.rename(columns={"Disease": "disease",
                                        "Precaution_1": "treatment1",
                                        "Precaution_2": "treatment2",
                                        "Precaution_3": "treatment3",
                                        "Precaution_4": "treatment4"})

# Add classifications column for disease severity - immediacy of seeking help based on if it says to go to hospital or consult a doctor


disease_df

### 2. Transform Diagnosis Sample Data

In [ ]:
# Create Sample ID
samples_df['sample_id'] = np.arange(samples_df.shape[0])

# Eliminate NaN
samples_df = samples_df.fillna('')
samples_df

In [ ]:
# Identify the number and list of unique symptoms
symptom_df = pd.melt(samples_df, id_vars=['sample_id','Disease'], value_vars=['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5', 'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10', 'Symptom_11', 'Symptom_12', 'Symptom_13', 'Symptom_14', 'Symptom_15', 'Symptom_16', 'Symptom_17'])
symptom_df

In [ ]:
#symptom_df = pd.lreshape(samples_df, {'Symptoms':['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5', 'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10', 'Symptom_11', 'Symptom_12', 'Symptom_13', 'Symptom_14', 'Symptom_15', 'Symptom_16', 'Symptom_17']})
pd.set_option('display.max_rows', 150)
symptom_df['value'].value_counts()

In [ ]:
unique_symptoms = symptom_df['value'].unique()
unique_symptoms

In [ ]:
symptom_df['weight'] = 1
symptom_df = symptom_df [['sample_id', 'Disease', 'value', 'weight']]
symptom_df = symptom_df.drop(symptom_df[symptom_df.value == ''].index)
symptom_df

In [ ]:
# Pivot the table to create the columns with binary values
pd.set_option("display.max_columns", None)
samples_transformed_df = symptom_df.reset_index().groupby(['sample_id', 'Disease', 'value'])['weight'].aggregate('first').unstack()
samples_transformed_df = samples_transformed_df.fillna(0)
samples_transformed_df

## Load 
##### Create Disease and Diagnosis Sample tables and load the dataframes for Disease and Diagnosis Sample data to the database.

In [ ]:
##### Create Database Connection #####

protocol = 'postgresql'
username = config.myusername
password = config.mypassword
host = 'localhost'
port = config.myport_number
database_name = 'diagnosis_db'

# create the database if it doesn't already exist
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string, echo = False)
if not database_exists(engine.url):
    create_database(engine.url)

# connect to the database
conn = engine.connect()

In [ ]:
##### Create Tables #####

# instantiate the base
Base = declarative_base()

# define tables
class diseases(Base):
    __tablename__ = "diseases"
    uid = Column(Integer, nullable = False, unique = True, primary_key = True)
    name = Column(String(50), nullable = False)

class diagnosis_samples(Base):
    __tablename__ = "diagnosis_samples"
    id = Column(Integer, nullable = False, unique = True, primary_key = True)
    disease = Column(String(250), nullable = False)
    
# create the tables
Base.metadata.create_all(engine)

In [ ]:
##### Load Data into the Tables #####

# confirm tables exist
inspector = inspect(engine)
inspector.get_table_names()

# Load Diseases to disease Table

# Load Diagnosis Samples to diagnosis_samples table